In [1]:
# Day_01_04_mnist.py
import tensorflow as tf
import numpy as np

from tensorflow.examples.tutorials.mnist import  input_data

In [2]:
def mnist_basic():
    np.set_printoptions(linewidth=1000)

    # 0~1사이의 mnist 값을 가지고 있는것,  다른 mnist데이터는 다르게 들어갈수 있다.
    mnist = input_data.read_data_sets('mnist')

    print(mnist.train.images.shape)       # (55000, 784)
    print(mnist.validation.images.shape)  # ( 5000, 784)
    print(mnist.test.images.shape)        # (10000, 784)

    print(mnist.train.labels.shape)       # (55000, )
    print(mnist.train.labels[:5])         # [7 3 4 6 1]
    # print(mnist.train.images[0])
    # pixel 기준으로 보게
    # print((mnist.train.images[0].reshape(28,28)))

# Stocastic Gradient Decent(SGD)
* 아래 소스는 weight를 다 돌고 나서 한번에 한다. 안좋은 방법.

In [3]:
def mnist_sgd():
    mnist = input_data.read_data_sets('mnist')

    # 784가 feature, 28*28 중에 1개의 pixel이 1의 feature가 된다.
    # w = tf.Variable(tf.random_uniform([784, 10]))
    # 모델 정확도 높이기 위해 아님  ->  빨리 수렴한다.
    # get_variable은 변수가 만들어져 있으면 가져오고 , 없으면 만든다.  glorot은  xavier
    w = tf.get_variable('w', shape=[784,10], initializer=tf.initializers.glorot_normal())
    # w = tf.Variable(tf.contrib.layers.xavier_initializer([784,10]))
    # bias는 class의 갯수만큼 정한다.
    # b = tf.Variable(tf.random_uniform([10]))
    # 정확도 올리기위해
    b = tf.Variable(tf.zeros([10]))

    ph_x = tf.placeholder(tf.float32)

    # (55000, 10) = (55000, 784) @  (784, 10)
    z = tf.matmul(ph_x, w) + b
    hx = tf.nn.softmax(z)

    # tensorflow 2.0d에서 정리 될것,  logit=예측  label을 결과괌
    # logit에 z를 하는것은 hx를 전달하지 않아도 됨, 자체적으로 함
    # 우리의 결과는 ont-hot 백터가 아님으로  sparse_softmax_cross_entropy_with_logits
    loss_i = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=z, labels=np.int32(mnist.train.labels))
    loss = tf.reduce_mean(loss_i)

    # 모델 정확도 높이기 위해 아님  ->  빨리 수렴한다.
    # optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train = optimizer.minimize(loss=loss)

    # sess로 변수의 값을 알수 있다.
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    for i in range(100):
        sess.run(train, feed_dict={ph_x: mnist.train.images})
        print(i, sess.run(loss, {ph_x: mnist.train.images}))

    preds = sess.run(hx, {ph_x: mnist.test.images})
    preds_arg = np.argmax(preds, axis=1)  # 1: 수평, 0: 수직
    # spase일때는 argmax를 가져올필요 없다.
    # test_arg  = np.argmax(mnist.test.labels, axis=1)

    # 파이썬의 list는 broadcasting기능이 없어서, numpy array로 변경
    # grades = np.array(['Setosa', 'Versicolor', 'Virginica'])
    # print(grades[preds_arg])
    # print(preds)

    # 1차 혼돈 : 데이터가 섞여 있지 않음으로 인한오류 → shuffle 필요 np.random.shuffle(iris)
    # 2차 혼돈 : 돌릴때 마다 위치가 달라져서 ...np.random.seed(1)
    print('acc: ', np.mean(preds_arg == mnist.test.labels))

    print(preds_arg)
    # print(test_arg)

    sess.close()

In [4]:
mnist_sgd()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist\train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting mnist\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting mnist\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Colocations handled automaticall

# 문제 1
* 테스트셋의 정확도를 알려주세요.
* hint : 데이터를 쪼개쓰는 mini_batch

In [13]:
def mnist_softmax_mini_batch():
    mnist = input_data.read_data_sets('mnist')

    # 784가 feature, 28*28 중에 1개의 pixel이 1의 feature가 된다.
    # w = tf.Variable(tf.random_uniform([784, 10]))
    # 모델 정확도 높이기 위해 아님  ->  빨리 수렴한다.
    # get_variable은 변수가 만들어져 있으면 가져오고 , 없으면 만든다.  glorot은  xavier
    w = tf.get_variable('w2', shape=[784,10], initializer=tf.initializers.glorot_normal())
    # w = tf.Variable(tf.contrib.layers.xavier_initializer([784,10]))
    # bias는 class의 갯수만큼 정한다.
    # b = tf.Variable(tf.random_uniform([10]))
    # 정확도 올리기위해
    b = tf.Variable(tf.zeros([10]))

    ph_x = tf.placeholder(tf.float32)
    ph_y = tf.placeholder(tf.int32)

    # (55000, 10) = (55000, 784) @  (784, 10)
    z = tf.matmul(ph_x, w) + b
    hx = tf.nn.softmax(z)

    # tensorflow 2.0d에서 정리 될것,  logit=예측  label을 결과괌
    # logit에 z를 하는것은 hx를 전달하지 않아도 됨, 자체적으로 함
    # 우리의 결과는 ont-hot 백터가 아님으로  sparse_softmax_cross_entropy_with_logits
    loss_i = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=z, labels=ph_y)
    loss = tf.reduce_mean(loss_i)

    # 모델 정확도 높이기 위해 아님  ->  빨리 수렴한다.
    # optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train = optimizer.minimize(loss=loss)

    # sess로 변수의 값을 알수 있다.
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    epochs = 10
    batch_size = 100   # 출구를 몇개씩 조사할건지
    n_iters = mnist.train.num_examples // batch_size # 550

    # epcochs를 1번 돌고 념
    for i in range(epochs):
        c = 0
        for j in range(n_iters):
            xx, yy = mnist.train.next_batch(batch_size)

            sess.run(train, feed_dict={ph_x: xx, ph_y: yy})
            c += sess.run(loss, {ph_x: xx, ph_y:yy})

        print(i, c/ n_iters)

    preds = sess.run(hx, {ph_x: mnist.test.images})
    preds_arg = np.argmax(preds, axis=1)  # 1: 수평, 0: 수직
    # spase일때는 argmax를 가져올필요 없다.
    # test_arg  = np.argmax(mnist.test.labels, axis=1)

    # 파이썬의 list는 broadcasting기능이 없어서, numpy array로 변경
    # grades = np.array(['Setosa', 'Versicolor', 'Virginica'])
    # print(grades[preds_arg])
    # print(preds)

    # 1차 혼돈 : 데이터가 섞여 있지 않음으로 인한오류 → shuffle 필요 np.random.shuffle(iris)
    # 2차 혼돈 : 돌릴때 마다 위치가 달라져서 ...np.random.seed(1)
    print('acc: ', np.mean(preds_arg == mnist.test.labels))

    print(preds_arg)
    # print(test_arg)

    sess.close()

In [14]:
mnist_softmax_mini_batch()

Extracting mnist\train-images-idx3-ubyte.gz
Extracting mnist\train-labels-idx1-ubyte.gz
Extracting mnist\t10k-images-idx3-ubyte.gz
Extracting mnist\t10k-labels-idx1-ubyte.gz
0 0.34581849932670594
1 0.28294095662507146
2 0.27262380918318574
3 0.2672042059221051
4 0.26281767536293377
5 0.26140581697225573
6 0.2604996431144801
7 0.25685677154497666
8 0.25420204677365044
9 0.25061307678845796
acc:  0.9208
[7 2 1 ... 4 5 6]


# 멀티 layer

In [15]:
def mnist_multi_layers():
    mnist = input_data.read_data_sets('mnist')

    w1 = tf.get_variable('w11', shape=[784, 512], initializer=tf.initializers.glorot_normal())
    w2 = tf.get_variable('w22', shape=[512, 256], initializer=tf.initializers.glorot_normal())
    w3 = tf.get_variable('w32', shape=[256,  10], initializer=tf.initializers.glorot_normal())
    b1 = tf.Variable(tf.zeros([512]))
    b2 = tf.Variable(tf.zeros([256]))
    b3 = tf.Variable(tf.zeros([ 10]))

    ph_x = tf.placeholder(tf.float32)
    ph_y = tf.placeholder(tf.int32)

    # (55000, 10) = (55000, 784) @  (784, 10)
    z1 = tf.matmul(ph_x, w1) + b1
    r1 = tf.nn.relu(z1)

    z2 = tf.matmul(r1, w2) + b2
    r2 = tf.nn.relu(z2)

    z3 = tf.matmul(r2, w3) + b3
    hx = tf.nn.softmax(z3)

    # tensorflow 2.0d에서 정리 될것,  logit=예측  label을 결과괌
    # logit에 z를 하는것은 hx를 전달하지 않아도 됨, 자체적으로 함
    # 우리의 결과는 ont-hot 백터가 아님으로  sparse_softmax_cross_entropy_with_logits
    loss_i = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=z3, labels=ph_y)
    loss = tf.reduce_mean(loss_i)

    # 모델 정확도 높이기 위해 아님  ->  빨리 수렴한다.
    # optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train = optimizer.minimize(loss=loss)

    # sess로 변수의 값을 알수 있다.
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    epochs = 10
    batch_size = 100   # 출구를 몇개씩 조사할건지
    n_iters = mnist.train.num_examples // batch_size # 550

    # epcochs를 1번 돌고 념
    for i in range(epochs):
        c = 0
        for j in range(n_iters):
            xx, yy = mnist.train.next_batch(batch_size)

            sess.run(train, feed_dict={ph_x: xx, ph_y: yy})
            c += sess.run(loss, {ph_x: xx, ph_y:yy})

        print(i, c/ n_iters)

    preds = sess.run(hx, {ph_x: mnist.test.images})
    preds_arg = np.argmax(preds, axis=1)  # 1: 수평, 0: 수직
    # spase일때는 argmax를 가져올필요 없다.
    # test_arg  = np.argmax(mnist.test.labels, axis=1)

    # 파이썬의 list는 broadcasting기능이 없어서, numpy array로 변경
    # grades = np.array(['Setosa', 'Versicolor', 'Virginica'])
    # print(grades[preds_arg])
    # print(preds)

    # 1차 혼돈 : 데이터가 섞여 있지 않음으로 인한오류 → shuffle 필요 np.random.shuffle(iris)
    # 2차 혼돈 : 돌릴때 마다 위치가 달라져서 ...np.random.seed(1)
    print('acc: ', np.mean(preds_arg == mnist.test.labels))

    print(preds_arg)
    # print(test_arg)

    sess.close()


In [16]:
mnist_multi_layers()

Extracting mnist\train-images-idx3-ubyte.gz
Extracting mnist\train-labels-idx1-ubyte.gz
Extracting mnist\t10k-images-idx3-ubyte.gz
Extracting mnist\t10k-labels-idx1-ubyte.gz
0 0.17192414816807616
1 0.08467089972712777
2 0.0689768023060804
3 0.056971493860368025
4 0.0562265362912281
5 0.04805349267225459
6 0.0424938825201984
7 0.043625768014674327
8 0.03891693604199893
9 0.032483553080528506
acc:  0.9728
[7 2 1 ... 4 5 6]
